# CNN (only convolution)

In [1]:
# Create simple model
from keras.layers import Conv2D
from keras.models import Sequential
import keras.backend as K


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 745 (CNMeM is disabled, cuDNN 5103)


In [2]:
model = Sequential()
model.add( Conv2D(16, 3, 3,activation='relu', border_mode='same', input_shape=(9,320, 480) ) )
model.add( Conv2D(32, 3,3, activation='relu', border_mode='same') )
model.add( Conv2D(1, 5,5, activation='sigmoid', border_mode='same') )

In [3]:
from keras.optimizers import Adam
from keras.metrics import binary_crossentropy

smooth = 1.
# From here: https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def bce_dice_loss(y_true, y_pred):
    return 0.5 * binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

model.compile(Adam(lr=1e-3), bce_dice_loss, metrics=['accuracy', dice_coef])

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 16, 320, 480)  1312        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 320, 480)  4640        convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 1, 320, 480)   801         convolution2d_2[0][0]            
Total params: 6,753
Trainable params: 6,753
Non-trainable params: 0
____________________________________________________________________________________________________


## Load Data Generator

In [6]:
import bcolz
X_val=bcolz.open('X_val')
y_val=bcolz.open('y_val')

In [8]:
def mygenerator():
    fileIndex=0
    st=0
    while 1:
        if st+25 < 5088:
            X = bcolz.open('X_new')[st:st+25] 
            X=X[:,:9,:,:]
            y=bcolz.open('y_new')[st:st+25]
            st+=0
        else:
            X = bcolz.open('X_new')[st:] 
            X=X[:,:9,:,:]
            y=bcolz.open('y_new')[st:]
            st+=0
        yield (X,y)
        

In [9]:
model.fit_generator(mygenerator(),samples_per_epoch=10,nb_epoch=1, validation_data=(X_val, y_val), verbose\
                    =2)

ValueError: Error when checking model input: expected convolution2d_input_1 to have shape (None, 9, 320, 480) but got array with shape (20, 320, 480, 9)

In [11]:
lst = [random.randint(lower, upper) for x in range(20)]

NameError: name 'random' is not defined